In [1]:
# --- Автоперезагрузка модулей ---
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import json
import pandas as pd
#pd.set_option('display.float_format', '{:.1f}'.format) ## 1 знак после запятой

In [5]:
from raw_data import RawData
from filtered_data import FilteredData
from raw_data_filter import RawDataFilter

In [7]:
# 1. Рабочая директория
work_dir = Path.cwd()
input_dir = work_dir / 'input'  # Теперь все данные ищем в папке input

# 2. Пути к файлам
gz_path = input_dir / '704.gz'

# 3. Создание объекта
if 'fRawData' in locals():
    del fRawData
fRawData = RawData(str(gz_path), user_id=704)

# 4. Получение сырых данных
raw_data = fRawData.get_raw_data()
#print(f"\n=== Сырые данные (первые 5 строк) данных {'704.gz'}===")
#print(raw_data.head(5))

In [9]:
def load_time_ranges(file_path, user_id):
    with open(file_path, 'r', encoding='utf-8') as f:
        all_timings = json.load(f)

    if str(user_id) not in all_timings:
        raise ValueError(f"Нет данных для пользователя {user_id} в файле {file_path}")

    return {item['shelf']: item['time_range'] for item in all_timings[str(user_id)]}

In [11]:
# Создаём FilteredData через процессор
fFilterData = RawDataFilter.create_filtered_data(fRawData)

# Фильтрация
RawDataFilter.filter_only_gaze(fFilterData)
RawDataFilter.filter_invalid_pupils(fFilterData, min_size=1., max_size=10.0)
RawDataFilter.filter_missing_data(fFilterData)
RawDataFilter.filter_by_timestamp(fFilterData,'01/01/2022') # filter by min dd-mm-yyyy
RawDataFilter.filter_unrealistic_event_spacing(fFilterData, 0.005) # filter by event rating 
RawDataFilter.filter_constant_pupil(fFilterData, 100, 0.001) # filter by constant pupil size
RawDataFilter.filter_async_pupil_size(fFilterData, 0.02) # filter by constant pupil size

In [13]:
# Вывод на экран
df = fFilterData.get_data()
#filtered_df = df[df['relative_timestamp'] > 558.8]
pd.set_option('display.float_format', '{:.2f}'.format) ## 2 знака после запятой
print(df.head(20))

       timestamp  type  eyeleft.pupildiameter  eyeright.pupildiameter  \
0  1720505684.56  gaze                   1.90                    1.89   
1  1720505684.58  gaze                   1.93                    1.90   
2  1720505684.60  gaze                   1.92                    1.89   
3  1720505684.62  gaze                   1.91                    1.89   
4  1720505684.64  gaze                   1.91                    1.88   
5  1720505684.66  gaze                   2.00                    1.88   
6  1720505684.68  gaze                   1.97                    1.88   
7  1720505684.70  gaze                   1.95                    1.88   
8  1720505684.72  gaze                   1.92                    1.89   
9  1720505684.74  gaze                   1.88                    1.86   
10 1720505684.76  gaze                   1.90                    1.86   
11 1720505684.78  gaze                   1.91                    1.86   
12 1720505684.80  gaze                   1.91      

In [ ]:
# Загружаем тайминги
#timings_path = input_dir / 'timings_by_shelf.json'
#time_ranges = load_time_ranges(timings_path, user_id=704)

#for shelf, (start_time, end_time) in time_ranges.items():
#    print(f"Shelf: {shelf}, Start: {start_time}, End: {end_time}")